In [1]:
# Library

import tkinter as tk
from tkinter import ttk
import pandas as pd
import re
import h5py
import numpy as np

# App window
win = tk.Tk()
win.title('NN_Classifier_GUI')  # Window Title

# Step 1: Import data frame name ---------------------------------

Name=ttk.Label(win,text="    Step 1: File Name")
Name.grid(row=0,column=0,sticky=tk.W)

Name_var=tk.StringVar()
Name_entrybox=ttk.Entry(win,width=16,textvariable=Name_var)
Name_entrybox.grid(row=0,column=1)

def Import_Data():
    global DB
    DF_Name=Name_var.get()
    DB_extension=re.findall("\..*", DF_Name) 
    if DB_extension==['.xlsx']:
        DB=pd.read_excel(DF_Name)
        confirm="Done"
    elif DB_extension==['.csv']:
        DB=pd.read_csv(DF_Name)
        confirm="Done"
    elif DB_extension==['.h5']:
        DB=h5py.File(DF_Name,'a')
    
        confirm="Done"
    else:
        confirm = "Not Done"
    
    # Blank empty window to print confirmation
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=0,column=3)
    Confirm_entrybox.insert(1,str(confirm))   

Import_Data_Button=ttk.Button(win,text="Import Data",command=Import_Data)
Import_Data_Button.grid(row=0,column=2)


# Step 2: Target data frame name ---------------------------------

Data=ttk.Label(win,text="    Step 2: Get Data")
Data.grid(row=1,column=0,sticky=tk.W)

def Get_Data():
    global DB, X_train, X_test, y_train, y_test, keys
    
    
    keys = DB.keys()
    x_test = np.array(DB.get("X_test"))
    x_train = np.array(DB.get("X_train"))
    x_val = np.array(DB.get("X_val"))
    y_test = np.array(DB.get("y_test"))
    y_train = np.array(DB.get ("y_train"))
    y_val = np.array(DB.get("y_val"))
    
    confirm="Got Data"
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=1,column=3)
    Confirm_entrybox.insert(1,str(confirm))


Data_Button=ttk.Button(win,text="Get Data",command=Get_Data)
Data_Button.grid(row=1,column=2)

# Step 3: NN_Model_Classifier ---------------------------------

def NNClassi():
    global DB,X,Y, model, XTRS, XTS, XVS, YTRC, YTC, YVC
    
    keys = DB.keys()
    x_test = np.array(DB.get("X_test"))
    x_train = np.array(DB.get("X_train"))
    x_val = np.array(DB.get("X_val"))
    y_test = np.array(DB.get("y_test"))
    y_train = np.array(DB.get ("y_train"))
    y_val = np.array(DB.get("y_val"))
    
    # Data Preprocessing
    
    #reshaping X data: (n, 32, 32) => (n, 1024)
    x_train = x_train.reshape((x_train.shape[0], -1))
    x_test = x_test.reshape((x_test.shape[0], -1))
    x_val = x_val.reshape((x_val.shape[0], -1))
    
    # Data Scaling
    from sklearn.preprocessing import StandardScaler as stdslr
    XTRS = stdslr().fit_transform(x_train)
    XTS = stdslr().fit_transform(x_test)
    XVS = stdslr().fit_transform(x_val)
   
    
    #Categorical conversion of Target Data
    from keras.utils.np_utils import to_categorical
    YTRC = to_categorical(y_train)
    YTC = to_categorical(y_test)
    YVC = to_categorical(y_val)
    
    from tensorflow.keras import Sequential # For Forward Propagation
    from tensorflow.keras.layers import Dense, LeakyReLU, Activation, Flatten
    from tensorflow.keras import optimizers
    from tensorflow.keras.layers import BatchNormalization, Dropout
    
    model = Sequential()
    
    model.add(Dense(100, input_shape = (1024, ), kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))    
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, kernel_initializer='he_normal'))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.summary()
    history = model.fit(XTRS, YTRC,validation_data= (XVS, YVC),
                    batch_size = 200, epochs = 50, verbose = 1)
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=6,column=2)
    Confirm_entrybox.insert(1,str("Network Trained"))

Cla_Button=ttk.Button(win,text="Train",command=NNClassi)
Cla_Button.grid(row=6,column=1)

Pickle=ttk.Label(win,text="Pickle")
Pickle.grid(row=4,column=0,sticky=tk.E)

def Pickle2():
    # Pickle model to JSON
    Classifier_model_json = model.to_json()
    with open("Classifier_model.json", "w") as json_file:
        json_file.write(Classifier_model_json)
    # Pickle weights to HDF5
    model.save_weights("Classifier_model.h5")
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=7,column=2)
    Confirm_entrybox.insert(1,str("Saved model to disk"))

Pickle_Button2=ttk.Button(win,text="Save",command=Pickle2)
Pickle_Button2.grid(row=7,column=1)

def Pickle3():
    from keras.models import model_from_json
    # load json and create model
    json_file = open('Classifier_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)   
    # load weights into new model
    loaded_model.load_weights("Classifier_model.h5")
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=8,column=2)
    Confirm_entrybox.insert(1,str("Loaded model to disk"))
    
Pickle_Button3=ttk.Button(win,text="Load",command=Pickle3)
Pickle_Button3.grid(row=8,column=1)

def Pickle4():
    from keras.models import model_from_json
    # load json and create model
    json_file = open('Classifier_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)   
    # load weights into new model
    loaded_model.load_weights("Classifier_model.h5")
    #Evaluate the Model
    loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    score = loaded_model.evaluate(XTS,YTC, verbose=0)
    #print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
    
    Confirm = score[1]*100
    Confirm_entrybox=ttk.Entry(win,width=20)
    Confirm_entrybox.grid(row=9,column=2)
    Confirm_entrybox.insert(1,str(Confirm))

Pickle_Button4=ttk.Button(win,text="Run",command=Pickle4)
Pickle_Button4.grid(row=9,column=1)    
    
win.mainloop()

/Users/shadab_019/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               102500    
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0

Epoch 39/50
210/210 [==============================] - 2s 10ms/step - loss: 0.6667 - accuracy: 0.7961 - val_loss: 0.4611 - val_accuracy: 0.8590
Epoch 40/50
210/210 [==============================] - 2s 10ms/step - loss: 0.6637 - accuracy: 0.7972 - val_loss: 0.4591 - val_accuracy: 0.8604
Epoch 41/50
210/210 [==============================] - 2s 10ms/step - loss: 0.6534 - accuracy: 0.8008 - val_loss: 0.4631 - val_accuracy: 0.8595
Epoch 42/50
210/210 [==============================] - 2s 11ms/step - loss: 0.6557 - accuracy: 0.7984 - val_loss: 0.4470 - val_accuracy: 0.8650
Epoch 43/50
210/210 [==============================] - 2s 10ms/step - loss: 0.6527 - accuracy: 0.8002 - val_loss: 0.4549 - val_accuracy: 0.8631
Epoch 44/50
210/210 [==============================] - 2s 10ms/step - loss: 0.6513 - accuracy: 0.8005 - val_loss: 0.4405 - val_accuracy: 0.8672
Epoch 45/50
210/210 [==============================] - 2s 10ms/step - loss: 0.6457 - accuracy: 0.8034 - val_loss: 0.4289 - val_accuracy: